# <center> N-Body gravitacional em Python x Python NumPy x </center>
# <center> Python Numba CPU x Python Numba CPU // x Python Numba GPU 64 bits </center>
## <center> v1.6.4 - 22/11/2018 a 22/02/2019 </center>
## <center> $M = 1$ passo, $dt = 0.01$, $N = 32^3 = 32K$ corpos em cubo homogêneo</center>

### Autoria da v1.0, v1.3, v1.4, 1.6 : Flávio Manoel Santos Hemerli
#### Revisão v1.1, v1.2, v1.3.1, v1.4.1, v1.4.6-9, v1.5.0-6, v1.6.2-4 : Roberto Colistete Júnior

Computador do prof. Roberto Colistete Jr. :  
**Notebook Dell XPS 15** com CPU [**Intel Core i7 2670QM**](https://ark.intel.com/products/53469/Intel-Core-i7-2670QM-Processor-6M-Cache-up-to-3-10-GHz-) 2,2-3,1 GHz 4 cores/8 threads 6MB cache com GPU integrada Intel HD Graphics 3000 & GPU dedicada [**NVidia GeForce GPU GT 540M**](https://www.geforce.com/hardware/notebook-gpus/geforce-gt-540m/specifications) 2GB 96 cores CC2.1.  
Manjaro KDE 18.0.2 64 bits, kernel 4.20.3, gcc 8.2.1, driver NVidia 390.87, CUDA 8.0.  
Anaconda 3 2018-12, Python 3.7.1, NumPy 1.15.4, PyCUDA 2018.1.1 (Python 2.7.15), Numba 0.41.0 com CUDAToolkit 8.0.

Computador do prof. Roberto Colistete Jr. :
**Notebook Dell G3 15 3579** com CPU [**Intel Core i7 8750H**](https://ark.intel.com/products/134906/Intel-Core-i7-8750H-Processor-9M-Cache-up-to-4-10-GHz-) Kaby Lake 2,2-4,1 GHz 6 cores/12 threads 9MB cache com GPU integrada Intel UHD Graphics 630 & GPU dedicada [**NVidia GeForce GTX 1050 Ti Mobile**](https://www.nvidia.com/en-us/geforce/products/10series/laptops/) 4GB 768 cores CC6.1.   
Manjaro KDE 18.0.2 64 bits, kernel 4.20.1, gcc 8.2.1, driver NVidia 415.25, CUDA 10.0.  
Anaconda 3 2018-12, Python 3.7.1, NumPy 1.15.4, PyCUDA 2018.1.1 (Python 2.7.15), Numba 0.41.0 com CUDAToolkit 9.2.

Computador do aluno Flávio Manoel Santos Hemerli:   
**Notebook Lenovo Ideapad 320** com CPU [**Intel Core i5 7200U**](https://ark.intel.com/pt-br/products/95443/Intel-Core-i5-7200U-Processor-3M-Cache-up-to-3-10-GHz-) 2,5-3,1 GHz 2 cores/4 threads 3MB cache com GPU integrada Intel HD Graphics 620. 8GB RAM DDR4 2133MHz Dual Channel.  
Manjaro Deepin 18.0 64 bits. Anaconda 3 2018-12, Python 3.7.1, NumPy 1.15.4, Numba 0.41.0

-> **Computador na nuvem Google Colab(oratory)** :  
Servidor com CPU [**Intel Xeon** E5-2699 (ou semelhante)](https://ark.intel.com/pt-br/products/81061/Intel-Xeon-Processor-E5-2699-v3-45M-Cache-2-30-GHz-) 2,3-3,6 GHz 18 cores/36 threads 45MB cache & GPU dedicada [**NVidia Tesla K80**](https://www.nvidia.com.br/object/tesla-k80-br.html) 24GB 2x2496 cores CC3.7.  
Acessível via Google Colab : Xeon 2 cores 12-13GB de RAM, K80 com (possivelmente) 2496 cores e 11GB de RAM.
Ubuntu 18.04.1 64 bits. Python 3.6.7, NumPy 1.14.6, Numba 0.40.1 com CUDA 9.2.

### Inicialização

In [1]:
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'

/usr/local/cuda-10.0/nvvm/libdevice
/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so


In [0]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.0/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so"

In [0]:
import math
import numpy as np
from timeit import default_timer as timer
import numba as nb
from numba import jit, njit, prange, cuda

### Informação sobre versão de Linux, CPU :

In [4]:
!uname -a

Linux e89ca8f71e52 4.14.79+ #1 SMP Wed Dec 19 21:19:13 PST 2018 x86_64 x86_64 x86_64 GNU/Linux


In [5]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.1 LTS
Release:	18.04
Codename:	bionic


In [6]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.000
BogoMIPS:            4400.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

Versão de Numba :

In [7]:
nb.__version__

'0.40.1'

In [8]:
nb.config.NUMBA_DEFAULT_NUM_THREADS

2

Versão de gcc :

In [9]:
!gcc --version

gcc (Ubuntu 7.3.0-27ubuntu1~18.04) 7.3.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



Versão de Python :

In [10]:
!python --version

Python 3.6.7


Versão de NumPy :

In [11]:
np.__version__

'1.14.6'

### Informação sobre GPU/CUDA :

In [12]:
print(cuda.gpus)

<Managed Device 0>


In [13]:
cuda.is_available()

True

In [14]:
cuda.detect()

Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

In [15]:
cuda.select_device(0).name

b'Tesla K80'

In [16]:
cuda.get_current_device().compute_capability

(3, 7)

In [17]:
cuda.current_context(1).get_memory_info()

_MemoryInfo(free=11849826304, total=11996954624)

## Definições de nbody para Python puro, NumPy, Numba CPU, Numba CPU // e Numba GPU

In [0]:
G = 6.67408*1e-11 # define a constante gravitacional
sft = 1e-10 # define a constante de atenuamento para tratamento das singularidades

In [0]:
@njit
def gravitationalNbody_NumbaCPU(r, v, a, m, nobj, ninter, dt, flag_Energy=False):
    for inter in range(ninter):
        for i in range(nobj):
            a[i][0] = 0.0; a[i][1] = 0.0; a[i][2] = 0.0
            for j in range(nobj):
                if i != j:
                    # calculate a_ij acceleration
                    dx = r[j][0] - r[i][0]; dy = r[j][1] - r[i][1]; dz = r[j][2] - r[i][2]
                    dsq = dx*dx + dy*dy + dz*dz + sft*sft
                    gA = (G*m[j])/(dsq*math.sqrt(dsq))
                    # sum a_ij_x to a_i_x, etc
                    a[i][0] += gA*dx; a[i][1] += gA*dy; a[i][2] += gA*dz
            # update the v_i velocity
            v[i][0] += a[i][0]*dt; v[i][1] += a[i][1]*dt; v[i][2] += a[i][2]*dt
        for i in range(nobj):
            # update the r_i position
            r[i][0] += v[i][0]*dt; r[i][1] += v[i][1]*dt; r[i][2] += v[i][2]*dt
    K = 0.0; U = 0.0
    if flag_Energy:
        for i in range(nobj):
            # Sum the total kinectic energy
            K += 0.5*m[i]*(v[i][0]**2 + v[i][1]**2 + v[i][2]**2)
            for j in range(nobj):
                if (i < j):
                    # Sum the total potential energy
                    U += (-G*m[i]*m[j])/math.sqrt((r[j][0] - r[i][0])**2 + (r[j][1] - r[i][1])**2 + 
                                                  (r[j][2] - r[i][2])**2 + sft**2)
    return K, U

In [0]:
@njit(parallel=True)
def gravitationalNbody_NumbaCPUparallel(r, v, a, m, nobj, ninter, dt, flag_Energy=False):
    for inter in range(ninter):
        for i in prange(nobj):
            a[i][0] = 0.0; a[i][1] = 0.0; a[i][2] = 0.0
            for j in range(nobj):
                if i != j:
                    # calculate a_ij acceleration
                    dx = r[j][0] - r[i][0]; dy = r[j][1] - r[i][1]; dz = r[j][2] - r[i][2]
                    dsq = dx*dx + dy*dy + dz*dz + sft*sft
                    gA = (G*m[j])/(dsq*math.sqrt(dsq))
                    # sum a_ij_x to a_i_x, etc
                    a[i][0] += gA*dx; a[i][1] += gA*dy; a[i][2] += gA*dz
            # update the v_i velocity
            v[i][0] += a[i][0]*dt; v[i][1] += a[i][1]*dt; v[i][2] += a[i][2]*dt
        for i in prange(nobj):
            # update the r_i position
            r[i][0] += v[i][0]*dt; r[i][1] += v[i][1]*dt; r[i][2] += v[i][2]*dt
    K = 0.0; U = 0.0
    if flag_Energy:
        for i in prange(nobj):
            # Sum the total kinectic energy
            K += 0.5*m[i]*(v[i][0]**2 + v[i][1]**2 + v[i][2]**2)
            for j in range(nobj):
                if (i < j):
                    # Sum the total potential energy
                    U += (-G*m[i]*m[j])/math.sqrt((r[j][0] - r[i][0])**2 + (r[j][1] - r[i][1])**2 + 
                                                  (r[j][2] - r[i][2])**2 + sft**2)
    return K, U

In [0]:
@njit
def gravitationalNbody_NumPy_NumbaCPU(r, v, a, m, nobj, ninter, dt, flag_Energy=False):
    for inter in range(ninter):
        # zero the a acceleration
        a = np.zeros((nobj, 3))
        for i in range(nobj):
            # update the a acceleration
            dr = r[i] - r
            dsq = np.sum(dr*dr, axis=1) + sft*sft
            gA = (G*m[i])/(dsq*np.sqrt(dsq))
            a += (dr.T*gA).T
        # update the v velocity
        v += a*dt
        # update the r position
        r += v*dt
    K = 0.0; U = 0.0
    if flag_Energy:
        # Sum the total kinetic energy
        K = np.sum(0.5*m*np.sum(v*v, axis=1))
        U = 0.0
        for i in range(nobj):
            for j in range(nobj):
                if (i < j):
                    # Sum the total potential energy
                    U += (-G*m[i]*m[j])/math.sqrt((r[j][0] - r[i][0])**2 + (r[j][1] - r[i][1])**2 + 
                                                  (r[j][2] - r[i][2])**2 + sft**2)
    return K, U

In [0]:
@njit(parallel=True)
def gravitationalNbody_NumPy_NumbaCPUparallel(r, v, a, m, nobj, ninter, dt, flag_Energy=False):
    for inter in range(ninter):
        # zero the a acceleration
        a = np.zeros((nobj, 3))
        for i in prange(nobj):
            # update the a acceleration
            dr = r[i] - r
            dsq = np.sum(dr*dr, axis=1) + sft*sft
            gA = (G*m[i])/(dsq*np.sqrt(dsq))
            a += (dr.T*gA).T
        # update the v velocity
        v += a*dt
        # update the r position
        r += v*dt
    K = 0.0; U = 0.0
    if flag_Energy:
        K = np.sum(0.5*m*np.sum(v*v, axis=1))
        U = 0.0
        for i in prange(nobj):
            for j in range(nobj):
                if (i < j):
                    # Sum the total potential energy
                    U += (-G*m[i]*m[j])/math.sqrt((r[j][0] - r[i][0])**2 + (r[j][1] - r[i][1])**2 + 
                                                  (r[j][2] - r[i][2])**2 + sft**2)
    return K, U

In [0]:
@cuda.jit("(float64[:,:], float64[:,:], float64[:,:], float64[:], int32, float64)")
def gravitationalNbody_a_v_ij_NumbaGPU64(r, v, a, m, nobj, dt):
    i = cuda.grid(1)
    if i < nobj:
        a[i][0] = 0.0; a[i][1] = 0.0; a[i][2] = 0.0
        for j in range(nobj):
            if i != j:
                dx = r[j][0] - r[i][0]; dy = r[j][1] - r[i][1]; dz = r[j][2] - r[i][2]
                dsq = dx*dx + dy*dy + dz*dz + sft*sft
                gA = (G*m[j])/(dsq*math.sqrt(dsq))
                # sum a_ij_x to a_i_x, etc
                a[i][0] += gA*dx; a[i][1] += gA*dy; a[i][2] += gA*dz
        v[i][0] += a[i][0]*dt; v[i][1] += a[i][1]*dt; v[i][2] += a[i][2]*dt

In [0]:
@cuda.jit("(float64[:,:], float64[:,:], int32, float64)")        
def gravitationalNbody_r_i_NumbaGPU64(r, v, nobj, dt):
    i = cuda.grid(1)
    if i < nobj:
        r[i][0] += v[i][0]*dt; r[i][1] += v[i][1]*dt; r[i][2] += v[i][2]*dt

In [0]:
@cuda.jit("(float64[:,:], float64[:,:], float64[:], int32, float64[:])")
def gravitationalNbody_K_U_NumbaCUDA(r, v, m, nobj, K_U):
    i = cuda.grid(1)
    if i < nobj:
        K_U[0] += 0.5*m[i]*(v[i][0]**2 + v[i][1]**2 + v[i][2]**2)
        for j in range(nobj):
            if (i < j):
                K_U[1] += (-G*m[i]*m[j])/math.sqrt((r[j][0] - r[i][0])**2 + (r[j][1] - r[i][1])**2 + 
                                              (r[j][2] - r[i][2])**2 + sft**2)

In [0]:
@jit(parallel=True)
def gravitationalNbody_NumbaGPU64(r, v, a, m, nobj, ninter, dt, flag_Energy=False):
    threadsperblock = 128
    blockspergrid = (nobj + (threadsperblock - 1)) // threadsperblock
    print("Threads per block = {}; Blocks per grid = {}".format(threadsperblock, blockspergrid))
    htod_ti = timer()
    d_r = cuda.to_device(r); d_v = cuda.to_device(v); d_a = cuda.to_device(a); d_m = cuda.to_device(m)
    htod_tf = timer()
    for inter in range(ninter):
        gravitationalNbody_a_v_ij_NumbaGPU64[blockspergrid, threadsperblock](d_r, d_v, d_a, d_m, nobj, dt)
        cuda.synchronize()
        gravitationalNbody_r_i_NumbaGPU64[blockspergrid, threadsperblock](d_r, d_v, nobj, dt)
        cuda.synchronize()
    dtoh_ti = timer()
    d_r.copy_to_host(r); d_v.copy_to_host(v); d_a.copy_to_host(a); d_m.copy_to_host(m)
    dtoh_tf = timer()
    K = 0.0; U = 0.0
    if flag_Energy:
        for i in prange(nobj):
            # Sum the total kinectic energy
            K += 0.5*m[i]*(v[i][0]**2 + v[i][1]**2 + v[i][2]**2)
            for j in range(nobj):
                if (i < j):
                    # Sum the total potential energy
                    U += (-G*m[i]*m[j])/math.sqrt((r[j][0] - r[i][0])**2 + (r[j][1] - r[i][1])**2 + 
                                                  (r[j][2] - r[i][2])**2 + sft**2)
    print("Copied data from CPU to GPU after {} s".format(htod_tf - htod_ti))
    print("GPU calculations done after {} s".format(dtoh_ti - htod_tf))
    print("Copied data from GPU to CPU after {} s".format(dtoh_tf - dtoh_ti))
    print("Copies CPU<->GPU and GPU calculations after {} s".format(dtoh_tf - htod_ti))
    return K, U
#    Energies_K_U = np.zeros(2)
#    if flag_Energy:
#        gravitationalNbody_K_U_NumbaCUDA[blockspergrid, threadsperblock](r, v, m, nobj, Energies_K_U)
#        cuda.synchronize()
#    return Energies_K_U[0], Energies_K_U[1]

In [0]:
# n_cube_edge : número de pontos l de cada aresta de um cubo
# ninter : M, número de interações
# dt : dt, intervalo entre interações
def run_nbody(gravitationalNbodyfunction, n_cube_edge=10, ninter = 5, dt = 0.01, 
              flag_array=True, flag_compile=True, flag_Energy= False, flag_verbose=False):
    nbody_version = "1.6.3"

    nobj = n_cube_edge**3 # define o número de objetos na simulação
    if flag_array:
        positions = np.zeros((nobj, 3))   # inicializa uma lista de N posições 3D, nulas inicialmente
        velocities = np.zeros_like(positions)   # inicializa uma lista de N velocidades 3D, nulas inicialmente
        accelerations = np.zeros_like(positions)   # inicializa uma lista de N acelerações 3D, nulas inicialmente
        masses = 1e9*np.ones(nobj)   # inicializa uma lista de N massas, aqui iguais a 10^6 kg
    else:
        positions = [[0., 0., 0.] for i in range(nobj)]   # inicializa uma lista de N de objetos para posição e massa, nulas inicialmente
        velocities = [[0., 0., 0.] for i in range(nobj)]   # inicializa uma lista de N velocidades 3D, nulas inicialmente
        accelerations = [[0., 0., 0.] for i in range(nobj)]   # inicializa uma lista de N acelerações 3D, nulas inicialmente
        masses = [1e9]*nobj   # inicializa uma lista de N massas, aqui iguais a 10^6 kg
    # Cria um cubo de l^3 pontos, onde cada aresta tem l pontos   
    scale = 1.0  # escala do cubo, com comprimento da aresta = scale*n_cube_edge
    n_half_cube_edge = int(n_cube_edge/2)
    for i in range(n_cube_edge):
        for j in range(n_cube_edge):
            for k in range(n_cube_edge):
                positions[k + j*n_cube_edge + i*n_cube_edge*n_cube_edge] = [(i - n_half_cube_edge)*scale, 
                                                                            (j - n_half_cube_edge)*scale,
                                                                            (k - n_half_cube_edge)*scale]                
    if flag_compile:
        # To Numba compile the function 1st time before calling with full data
        positions_backup = positions.copy()
        velocities_backup = velocities.copy()
        accelerations_backup = accelerations.copy()
        print("N-Body (C) Flavio Manoel, v{}".format(nbody_version))
        print("N = {} corpos, dt = {}s, M = {} interações".format(nobj, dt, ninter))
        start = timer()
        Kf, Uf = gravitationalNbodyfunction(positions, velocities, accelerations, masses, nobj, ninter, dt, 
                                            flag_Energy=flag_Energy)
        end = timer()
        if flag_Energy:
            print("Após {} passos, {:.4f}s, Ef = {}, Kf = {}, Uf = {}".format(ninter, ninter*dt, 
                                                                                          Kf + Uf, Kf, Uf))
        dt_compile_run = end - start
        print("Código executado em {:.6f} s\n".format(dt_compile_run))
        positions = positions_backup.copy()
        velocities = velocities_backup.copy()
        accelerations = accelerations_backup.copy()
    else:
        dt_compile_run = 0.0
        
    print("N-Body (C) Flavio Manoel, v{}".format(nbody_version))
    print("N = {} corpos, dt = {}s, M = {} interações".format(nobj, dt, ninter))
    if flag_verbose:
        print("Posições = {}\nVelocidades = {}\nAcelerações = {}\nMassas = {}\n".format(positions, 
              velocities, accelerations, masses))
    start = timer()
    if flag_Energy:
        Ki, Ui = gravitationalNbodyfunction(positions, velocities, accelerations, masses, nobj, 0, dt, 
                                            flag_Energy=flag_Energy)
        print("Ei = {}, Ki = {}, Ui = {}".format(Ki + Ui, Ki, Ui))
    Kf, Uf = gravitationalNbodyfunction(positions, velocities, accelerations, masses, nobj, ninter, dt, 
                                        flag_Energy=flag_Energy)
    end = timer()
    if flag_Energy:
        erro_perc = 100.0*(Kf + Uf - (Ki + Ui))/(Ki + Ui)
        print("Após {} passos, {:.4f}s :\nEf = {}, Kf = {}, Uf = {}, erro em E = {}%".format(ninter, 
               ninter*dt, Kf + Uf, Kf, Uf, erro_perc))
    else:
        erro_perc = 0.0
    dt_run = end - start
    print("Código executado em {:.6f} s".format(dt_run))
    if flag_verbose:
        print("\nPositions = {}\nVelocities = {}\nAccelerations = {}".format(positions, velocities, 
                                                                             accelerations))
    return dt_compile_run, dt_run, erro_perc

## Testes de nbody

In [0]:
n_cube_edge=32; ninter = 1; dt = 0.01

### Teste de nbody em Python puro

In [0]:
#t_all_py, t_py, erro_py = run_nbody(gravitationalNbody_NumbaCPU.py_func, n_cube_edge=n_cube_edge, ninter=ninter, 
#                                    dt=dt, flag_array=False, flag_Energy=True, flag_compile=False)

In [30]:
t_all_py, t_py, erro_py = run_nbody(gravitationalNbody_NumbaCPU.py_func, n_cube_edge=n_cube_edge, ninter=ninter, 
                                    dt=dt, flag_array=False, flag_compile=False)

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Código executado em 1124.351219 s


### Teste de nbody em Python (com NumPy arrays)

In [0]:
#t_all_py_a, t_py_a, erro_py_a = run_nbody(gravitationalNbody_NumbaCPU.py_func, n_cube_edge=n_cube_edge, 
#                                          ninter=ninter, dt=dt, flag_Energy=True, flag_compile=False)

In [0]:
#t_all_py_a, t_py_a, erro_py_a = run_nbody(gravitationalNbody_NumbaCPU.py_func, n_cube_edge=n_cube_edge, 
#                                          ninter=ninter, dt=dt, flag_compile=False)

### Teste de nbody em Python NumPy

In [0]:
#t_all_np, t_np, erro_np = run_nbody(gravitationalNbody_NumPy_NumbaCPU.py_func, n_cube_edge=n_cube_edge, 
#                                    ninter=ninter, dt=dt, flag_Energy=True, flag_compile=False)

In [34]:
t_all_np, t_np, erro_np = run_nbody(gravitationalNbody_NumPy_NumbaCPU.py_func, n_cube_edge=n_cube_edge, 
                                    ninter=ninter, dt=dt, flag_compile=False)

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Código executado em 52.851517 s


### Teste de nbody em Python Numba CPU

In [0]:
#t_all_nb, t_nb, erro_nb = run_nbody(gravitationalNbody_NumbaCPU, n_cube_edge=n_cube_edge, ninter=ninter, 
#                                    dt=dt, flag_Energy=True)

In [36]:
t_all_nb, t_nb, erro_nb = run_nbody(gravitationalNbody_NumbaCPU, n_cube_edge=n_cube_edge, ninter=ninter, 
                                    dt=dt)

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Código executado em 10.808175 s

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Código executado em 10.242123 s


### Teste de nbody em NumPy Numba CPU

In [0]:
#t_all_np_nb, t_np_nb, erro_np_nb = run_nbody(gravitationalNbody_NumPy_NumbaCPU, n_cube_edge=n_cube_edge, 
#                                             ninter=ninter, dt=dt, flag_Energy=True)

In [38]:
t_all_np_nb, t_np_nb, erro_np_nb = run_nbody(gravitationalNbody_NumPy_NumbaCPU, n_cube_edge=n_cube_edge, 
                                             ninter=ninter, dt=dt)

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Código executado em 41.263902 s

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Código executado em 39.379604 s


### Teste de nbody em Python Numba CPU //

In [0]:
#t_all_nb_parallel, t_nb_parallel, erro_nb_parallel = run_nbody(gravitationalNbody_NumbaCPUparallel, 
#                                             n_cube_edge=n_cube_edge, ninter=ninter, dt=dt, flag_Energy=True)

In [40]:
t_all_nb_parallel, t_nb_parallel, erro_nb_parallel = run_nbody(gravitationalNbody_NumbaCPUparallel, 
                                                               n_cube_edge=n_cube_edge, ninter=ninter, dt=dt)

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Código executado em 13.115846 s

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Código executado em 9.477498 s


### Teste de nbody em NumPy Numba CPU //

In [0]:
#t_all_np_nb_parallel, t_np_nb_parallel, erro_np_nb_parallel = run_nbody(
#    gravitationalNbody_NumPy_NumbaCPUparallel, n_cube_edge=n_cube_edge, ninter=ninter, dt=dt, flag_Energy=True)

In [42]:
t_all_np_nb_parallel, t_np_nb_parallel, erro_np_nb_parallel = run_nbody(
                    gravitationalNbody_NumPy_NumbaCPUparallel, n_cube_edge=n_cube_edge, ninter=ninter, dt=dt)

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Código executado em 31.816481 s

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Código executado em 25.358395 s


### Teste de nbody em Python Numba GPU 64 bits (double precision)

In [0]:
#t_all_gpu, t_gpu, erro_gpu = run_nbody(gravitationalNbody_NumbaGPU64, n_cube_edge=n_cube_edge, 
#                                       ninter=ninter, dt=dt, flag_Energy=True)

In [44]:
t_all_gpu, t_gpu, erro_gpu = run_nbody(gravitationalNbody_NumbaGPU64, n_cube_edge=n_cube_edge, 
                                       ninter=ninter, dt=dt)

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Threads per block = 128; Blocks per grid = 256
Copied data from CPU to GPU after 0.0031563600005029002 s
GPU calculations done after 0.9581256860001304 s
Copied data from GPU to CPU after 0.00093545999880007 s
Copies CPU<->GPU and GPU calculations after 0.9622175059994333 s
Código executado em 1.906233 s

N-Body (C) Flavio Manoel, v1.6.3
N = 32768 corpos, dt = 0.01s, M = 1 interações
Threads per block = 128; Blocks per grid = 256
Copied data from CPU to GPU after 0.0019011059994227253 s
GPU calculations done after 0.6308841870013566 s
Copied data from GPU to CPU after 0.0009364859997731401 s
Copies CPU<->GPU and GPU calculations after 0.6337217790005525 s
Código executado em 0.635123 s


### Resultados e comparações

Speed up of pure Python with respect to Python (with NumPy arrays)  :

In [0]:
#t_py_a / t_py

Speed up of Python NumPy with respect to pure Python :

In [46]:
t_py / t_np

21.273773628330893

Speed up of Python Numba CPU (serial) with respect to pure Python :

In [47]:
t_py / t_nb

109.7771663110294

Speed up of Python Numba CPU (serial) with respect to Python (with NumPy arrays) :

In [0]:
#t_py_a / t_nb

Speed up of Python Numba CPU (serial) with respect to Python NumPy :

In [49]:
t_np / t_nb

5.160211264297558

Speed up of NumPy Numba CPU (serial) with respect to pure Python :

In [50]:
t_py / t_np_nb

28.551613375752158

Speed up of NumPy Numba CPU (serial) with respect to Python NumPy :

In [51]:
t_np / t_np_nb

1.3421038446009013

Speed up of Python Numba CPU // with respect to pure Python :

In [52]:
t_py / t_nb_parallel

118.63376528434956

Speed up of Python Numba CPU // with respect to Python (with NumPy arrays) :

In [0]:
#t_py_a / t_nb_parallel

Speed up of NumPy Numba CPU // with respect to Python NumPy :

In [54]:
t_np / t_np_nb_parallel

2.0841822824632086

Speed up of Python Numba CPU // with respect to Python Numba CPU :

In [55]:
t_nb / t_nb_parallel

1.0806779703916474

Speed up of NumPy Numba CPU // with respect to NumPy Numba CPU :

In [56]:
t_np_nb / t_np_nb_parallel

1.5529217734138727

Speed up of Python Numba GPU 64 bits with respect to pure Python :

In [57]:
t_py / t_gpu

1770.288970440466

Speed up of Python Numba GPU 64 bits with respect to Python (with NumPy arrays) :

In [0]:
#t_py_a / t_gpu

Speed up of NumPy Numba GPU 64 bits  with respect to Python NumPy :

In [59]:
t_np / t_gpu

83.21461915355354

Speed up of Python Numba GPU 64 bits  with respect to Python Numba CPU (serial) :

In [60]:
t_nb / t_gpu

16.126203926823383

Speed up of NumPy Numba GPU 64 bits with respect to Python Numba CPU // :

In [61]:
t_nb_parallel / t_gpu

14.92230282160661